# Outline

1. Getting data
2. Data cleaning & featurizing
3. Data Export

In [3]:
%matplotlib inline

import quandl as Quandl
Quandl.ApiConfig.api_key = "EGdC1RASF31yDGeBDRt7"
import numpy as np
import pandas as pd
import math
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib import style
import datetime
style.use('ggplot')

/Library/Python/2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## <font color = blue> Getting Data</font>

* Individual Stocks
* Market & Industry Indicators

### Individual Stocks

Scraping the top 200 tech companies from NASDAQ (https://www.nasdaq.com/screening/companies-by-industry.aspx?industry=Technology.

In [7]:
def fetch_prices(ticker_list, limit=None):
    '''
    takes in ticker list of strings and pulls in their data from quandl
    can limit number of stocks to query with limit param
    '''
    if limit:
        ticker_list = ticker_list[:limit]
        
    out_dict = {}
    for ticker_str in ticker_list:
    #     print(ticker_str)
        try:
            ticker_df = Quandl.get(ticker_str,api_key="EGdC1RASF31yDGeBDRt7")
            out_dict[ticker_str] = ticker_df
        except: 
            print(ticker_str + " does not exist on Quandl's side.")
    return out_dict

In [8]:
tech_tickers = np.array([a.strip().upper() for a in pd.read_csv('companylist.csv')['Symbol']])
quandl_codes_tech_tickers = ["WIKI/"+ticker for ticker in tech_tickers]

Get the ticker data!

In [5]:
tech_data = fetch_prices(quandl_codes_tech_tickers, limit=10)

WIKI/VNET does not exist on Quandl's side.
WIKI/JOBS does not exist on Quandl's side.
WIKI/WUBA does not exist on Quandl's side.
WIKI/ACIA does not exist on Quandl's side.


## <font color = blue>Data Cleaning & Featurizing </font>

* Cleaning
* Featurizing

Now, to forecast out, we need some data. We decided that we're forecasting out 1% of the data, thus we will want to, or at least *can* generate forecasts for each of the final 1% of the dataset. So when can we do this? When would we identify that data? We could call it now, but consider the data we're trying to forecast is not scaled like the training data was. Okay, so then what? Do we just do preprocessing.scale() against the last 1%? The scale method scales based on all of the known data that is fed into it. Ideally, you would scale both the training, testing, AND forecast/predicting data all together. Is this always possible or reasonable? No. If you can do it, you should, however. In our case, right now, we can do it. Our data is small enough and the processing time is low enough, so we'll preprocess and scale the data all at once.

In many cases, you wont be able to do this. Imagine if you were using gigabytes of data to train a classifier. It may take days to train your classifier, you wouldn't want to be doing this every...single...time you wanted to make a prediction. Thus, you may need to either NOT scale anything, or you may scale the data separately. As usual, you will want to test both options and see which is best in your specific case.

In [11]:
clean_dfs = {}
train_test_data = {}
for df, ticker in list(zip(tech_data.values(),tech_data.keys())):
    curr_clean_df = clean_df(df)
    clean_dfs[ticker] = curr_clean_df
    X,y,X_lately = prep_naive_train_test(curr_clean_df)
    train_test_data[ticker] = (X,y,X_lately)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
